In [7]:
# ---- Install and Import Necessary Libraries ----
!pip install datasets pandas numpy scikit-learn statsmodels nltk tqdm tensorflow

import pandas as pd
import numpy as np
from datasets import load_dataset
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import warnings
from tqdm import tqdm
import logging
import nltk

# ---- Configure Logging ----
logging.basicConfig(
    level=logging.INFO,
    filename='model_evaluation.log',
    filemode='w',
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# ---- Suppress Warnings ----
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

# ---- Download Necessary NLTK Data ----
nltk.download('vader_lexicon')

# ---- Load the Dataset ----
print("Loading the dataset...")
logging.info("Loading the dataset...")
ds = load_dataset("nbettencourt/SC454k")
df = pd.DataFrame(ds['train'])

# Display initial data
print("\nInitial DataFrame Head:")
print(df.head())
print("\nDataFrame Columns:")
print(df.columns)
logging.info(f"Initial DataFrame Head:\n{df.head()}")
logging.info(f"DataFrame Columns:\n{df.columns}")

# ---- Data Preprocessing ----

# Handle Missing Values
# Drop rows where the target 'weighted_avg_720_hrs' or 'weighted_avg_0_hrs' is missing or zero
df = df.dropna(subset=['weighted_avg_720_hrs', 'weighted_avg_0_hrs'])
df = df[df['weighted_avg_0_hrs'] != 0]

# Parse 'Date' Column
# Remove ' ET' from 'Date' strings if present and parse to datetime
df['Date_cleaned'] = df['Date'].astype(str).str.replace(' ET', '', regex=False)
df['Date'] = pd.to_datetime(df['Date_cleaned'], errors='coerce')
df = df.drop(columns=['Date_cleaned'])

# Drop rows with unparsed dates
df = df.dropna(subset=['Date'])

# Convert 'Date' to Unix Timestamp
# Convert 'Date' to Unix timestamp (seconds since epoch)
df['Date_unix'] = df['Date'].astype('int64') // 10**9

# Verify 'Date_unix' creation
print("\nDataFrame with 'Date_unix':")
print(df[['Date', 'Date_unix']].head())
logging.info(f"DataFrame with 'Date_unix':\n{df[['Date', 'Date_unix']].head()}")

# 6.4: Feature Selection
# Define numeric and categorical features
numeric_features = [
    'weighted_avg_-96_hrs',
    'weighted_avg_-48_hrs',
    'weighted_avg_-24_hrs',
    'weighted_avg_0_hrs',
    'Risk_Free_Rate',
    'Date_unix'
]

# Add 'Author' and 'Publication' to categorical features
categorical_features = ['Symbol', 'Sector', 'Industry', 'Author', 'Publication']

# Define text features
text_features = ['Title', 'Article']

target = 'weighted_avg_720_hrs'

required_columns = numeric_features + categorical_features + text_features + [target]
missing_cols = set(required_columns) - set(df.columns)
if missing_cols:
    print(f"\nMissing columns in DataFrame: {missing_cols}")
    logging.warning(f"Missing columns in DataFrame: {missing_cols}")
else:
    print("\nAll required columns are present.")
    logging.info("All required columns are present.")

# Remove Sentiment Scoring
# Sentiment scoring is removed as it did not improve performance

# ---- Train-Test Split ----
# Sort the DataFrame by 'Date' to maintain chronological order
df = df.sort_values('Date').reset_index(drop=True)

# Determine split indices
total_length = len(df)
train_size = int(total_length * 0.70)  # 70% for training
test_size = int(total_length * 0.15)   # 15% for testing
# update_size = total_length - train_size - test_size  # 15% for updating (not used here)

# Split the data
train_df = df.iloc[:train_size].reset_index(drop=True)
test_df = df.iloc[train_size:train_size + test_size].reset_index(drop=True)

print(f"\nTraining Set Size: {train_df.shape}")
print(f"Testing Set Size: {test_df.shape}")
logging.info(f"Training Set Size: {train_df.shape}")
logging.info(f"Testing Set Size: {test_df.shape}")

# Define training and testing features and targets
X_train = train_df[numeric_features + categorical_features + text_features].copy()
y_train = train_df[target].copy()

X_test = test_df[numeric_features + categorical_features + text_features].copy()
y_test = test_df[target].copy()

# ---- Build the Preprocessing Pipeline ----
# Preprocessing for numeric features: Standard Scaling
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Preprocessing for categorical features: One-Hot Encoding
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Preprocessing for text features: TF-IDF Vectorization
title_transformer = Pipeline(steps=[
    ('tfidf', TfidfVectorizer(max_features=500))
])

article_transformer = Pipeline(steps=[
    ('tfidf', TfidfVectorizer(max_features=1000))
])

# Combine preprocessing for numeric, categorical, and text features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('title_tfidf', title_transformer, 'Title'),
        ('article_tfidf', article_transformer, 'Article')
    ])

# ---- Create and Train the Linear Regression Model ----
# Create the Pipeline with preprocessing and Linear Regression
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Fit the model on the training data
print("\nTraining the Linear Regression model...")
logging.info("Training the Linear Regression model...")
model.fit(X_train, y_train)
print("Model training completed.")
logging.info("Model training completed.")

# ---- Make Predictions and Evaluate the Model ----
# Make predictions on the test set
print("\nMaking predictions on the test set...")
logging.info("Making predictions on the test set...")
y_pred = model.predict(X_test)

# ---- Compute Additional Metrics ----

# Calculate Actual and Predicted Returns
# 'weighted_avg_0_hrs' is the current price at the time of the article's release
test_df['Actual_Return'] = (test_df['weighted_avg_720_hrs'] / test_df['weighted_avg_0_hrs']) - 1
test_df['Predicted_Return'] = (y_pred / test_df['weighted_avg_0_hrs']) - 1

# Remove any remaining NaN or infinite values
test_df = test_df.replace([np.inf, -np.inf], np.nan).dropna(subset=['Actual_Return', 'Predicted_Return'])

# Calculate Overall MSE, RMSE, and R2 on Returns
overall_mse_returns = mean_squared_error(test_df['Actual_Return'], test_df['Predicted_Return'])
overall_rmse_returns = np.sqrt(overall_mse_returns)
overall_r2_returns = r2_score(test_df['Actual_Return'], test_df['Predicted_Return'])

print("\nLinear Regression Model Performance on Returns:")
print(f"Mean Squared Error (MSE): {overall_mse_returns:.4f}")
print(f"Root Mean Squared Error (RMSE): {overall_rmse_returns:.4f}")
print(f"R-squared (R2 ): {overall_r2_returns:.4f}")

logging.info("Linear Regression Model Performance on Returns:")
logging.info(f"Mean Squared Error (MSE): {overall_mse_returns:.4f}")
logging.info(f"Root Mean Squared Error (RMSE): {overall_rmse_returns:.4f}")
logging.info(f"R-squared (R2 ): {overall_r2_returns:.4f}")

# Define Strategy Return
# Strategy: Long position if Predicted_Return > 0, else no position
test_df['Strategy_Return'] = np.where(test_df['Predicted_Return'] > 0, test_df['Actual_Return'], 0)

# Sector Metrics (MSE, R2, and Sharpe Ratio per Sector)
sector_metrics = {}

for sector in test_df['Sector'].unique():
    sector_data = test_df[test_df['Sector'] == sector].copy()
    sector_data = sector_data.dropna(subset=['Actual_Return', 'Predicted_Return'])
    if len(sector_data) == 0:
        sector_metrics[sector] = {'mse': np.nan, 'r2': np.nan, 'sharpe': np.nan}
    else:
        mse_sector = mean_squared_error(sector_data['Actual_Return'], sector_data['Predicted_Return'])
        r2_sector = r2_score(sector_data['Actual_Return'], sector_data['Predicted_Return'])

        # Calculate Sharpe Ratio
        strategy_excess_return = sector_data['Strategy_Return'] - sector_data['Risk_Free_Rate']
        if strategy_excess_return.std() != 0:
            sharpe_ratio = strategy_excess_return.mean() / strategy_excess_return.std()
        else:
            sharpe_ratio = np.nan  # Undefined if no variation

        sector_metrics[sector] = {'mse': mse_sector, 'r2': r2_sector, 'sharpe': sharpe_ratio}

# Convert sector metrics to DataFrame
sector_metrics_df = pd.DataFrame.from_dict(sector_metrics, orient='index').reset_index()
sector_metrics_df = sector_metrics_df.rename(columns={'index': 'Sector'})
print("\nSector-wise Metrics (on Returns):")
print(sector_metrics_df)

logging.info("Sector-wise Metrics (on Returns):")
logging.info(f"{sector_metrics_df}")

# Overall Trend Accuracy
# Calculate the direction of actual and predicted returns
test_df['Actual_Direction'] = np.where(test_df['Actual_Return'] > 0, 1, 0)
test_df['Predicted_Direction'] = np.where(test_df['Predicted_Return'] > 0, 1, 0)

# Calculate accuracy of direction predictions
overall_trend_acc = (test_df['Actual_Direction'] == test_df['Predicted_Direction']).mean()

print(f"\nOverall Trend Accuracy: {overall_trend_acc:.2%}")
logging.info(f"Overall Trend Accuracy: {overall_trend_acc:.2%}")

# Sharpe Ratio
# Assuming 'Risk_Free_Rate' is per period (e.g., monthly)
# Calculate excess returns for the strategy
test_df['Strategy_Excess_Return'] = test_df['Strategy_Return'] - test_df['Risk_Free_Rate']

# Calculate Sharpe Ratio
# Handle case where standard deviation is zero
if test_df['Strategy_Excess_Return'].std() != 0:
    sharpe_ratio = test_df['Strategy_Excess_Return'].mean() / test_df['Strategy_Excess_Return'].std()
else:
    sharpe_ratio = np.nan  # Undefined if no variation

print(f"Sharpe Ratio: {sharpe_ratio:.4f}")
logging.info(f"Sharpe Ratio: {sharpe_ratio:.4f}")

# Sortino Ratio
# Define target return as 0 for downside deviation
target_return = 0
downside_returns = test_df['Strategy_Return'][test_df['Strategy_Return'] < target_return]
downside_deviation = downside_returns.std()

# Handle case where downside_deviation is zero
if downside_deviation != 0:
    sortino_ratio = (test_df['Strategy_Return'].mean() - test_df['Risk_Free_Rate'].mean()) / downside_deviation
else:
    sortino_ratio = np.nan  # Undefined if no downside deviation

print(f"Sortino Ratio: {sortino_ratio:.4f}")
logging.info(f"Sortino Ratio: {sortino_ratio:.4f}")

# Average Return
average_return = test_df['Strategy_Return'].mean()

print(f"Average Strategy Return: {average_return:.4f}")
logging.info(f"Average Strategy Return: {average_return:.4f}")

# Win Rate
# Calculate the proportion of profitable strategy returns
wins = test_df['Strategy_Return'] > 0
win_rate = wins.mean()

print(f"Win Rate: {win_rate:.2%}")
logging.info(f"Win Rate: {win_rate:.2%}")

# Profit Factor
# Calculate gross profits and gross losses
gross_profit = test_df.loc[test_df['Strategy_Return'] > 0, 'Strategy_Return'].sum()
gross_loss = -test_df.loc[test_df['Strategy_Return'] < 0, 'Strategy_Return'].sum()

# Handle division by zero
if gross_loss != 0:
    profit_factor = gross_profit / gross_loss
else:
    profit_factor = np.inf  # Infinite profit factor if no losses

print(f"Profit Factor: {profit_factor:.4f}")
logging.info(f"Profit Factor: {profit_factor:.4f}")

# ---- Print and Log Per-Sector Metrics ----
print("\nPer-Sector Metrics:")
logging.info("Per-Sector Metrics:")

for index, row in sector_metrics_df.iterrows():
    sector = row['Sector']
    mse_sector = row['mse']
    r2_sector = row['r2']
    sharpe_sector = row['sharpe']

    # Print sector metrics
    print(
        f"Sector: {sector} - "
        f"MSE: {mse_sector:.4f}, "
        f"R²: {r2_sector:.4f}, "
        f"Sharpe Ratio: {sharpe_sector:.4f}"
    )

    # Log sector metrics
    logging.info(
        f"Sector: {sector} - "
        f"MSE: {mse_sector:.4f}, "
        f"R²: {r2_sector:.4f}, "
        f"Sharpe Ratio: {sharpe_sector:.4f}"
    )


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Loading the dataset...


Repo card metadata block was not found. Setting CardData to empty.



Initial DataFrame Head:
  Symbol                            Security                  Sector  \
0    UTL                  Unitil Corporation               Utilities   
1     HE  Hawaiian Electric Industries, Inc.               Utilities   
2     MG                 Mistras Group, Inc.  Consumer Discretionary   
3    GNK    Genco Shipping & Trading Limited  Consumer Discretionary   
4     BB                  BlackBerry Limited              Technology   

                                  Industry  \
0                         Power Generation   
1              Electric Utilities: Central   
2            Military/Government/Technical   
3                    Marine Transportation   
4  Computer Software: Prepackaged Software   

                                                 URL                 Date  \
0  https://www.nasdaq.com/articles/safest-high-yi...  2009-10-09 11:48:00   
1  https://www.nasdaq.com/articles/safest-high-yi...  2009-10-09 11:48:00   
2  https://www.nasdaq.com/articles